In [ ]:
# Install required packages
!pip install streamlit pandas openpyxl tabula-py PyMuPDF pytesseract pdf2image Pillow opencv-python-headless
!apt-get install poppler-utils tesseract-ocr
!pip install pyngrok

# Save the code to a file
code = """
import pandas as pd
import tabula
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image, ImageFile
import numpy as np
import cv2
import os
import re
import openpyxl
from openpyxl.styles import Font, Alignment
import streamlit as st

# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Directory to save images and PDFs
save_directory = 'pdf_images/'

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Function to detect images
def detect_images(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w > 50 and h > 50:  # Filter out small boxes that are likely not images
            image_roi = image[y:y+h, x:x+w]
            image_name = f"{save_directory}detected_image_{x}_{y}.jpg"
            cv2.imwrite(image_name, image_roi)
            images.append(image_name)

    return images

# Function to extract key-value pairs from text
def extract_key_value_pairs(text):
    lines = text.split('\\n')
    data = {}
    key = None
    for line in lines:
        if ':' in line:
            parts = line.split(':', 1)
            key = parts[0].strip()
            value = parts[1].strip()
            if key and value:
                data[key] = value
        elif key:
            # Handling multiline values
            data[key] += ' ' + line.strip()
    return data

# Function to save images as PDF
def save_images_as_pdf(image_paths):
    pdf_files = []
    for image_path in image_paths:
        if os.path.exists(image_path):
            image = Image.open(image_path)
            pdf_path = image_path.replace(".jpg", ".pdf")
            image.save(pdf_path, "PDF", resolution=100.0)
            pdf_files.append(pdf_path)
        else:
            print(f"Warning: {image_path} does not exist.")
    return pdf_files

# Streamlit app
st.title("PDF Table and Data to Excel Converter")
st.write("Upload a PDF file to convert its table and other data to an Excel file.")

uploaded_files = st.file_uploader("Choose PDF files", type="pdf", accept_multiple_files=True)

if uploaded_files:
    combined_data_list = []
    image_counter = 1
    image_data_list = []

    for uploaded_file in uploaded_files:
        # Save uploaded file to disk
        pdf_path = f"{save_directory}uploaded_{uploaded_file.name}"
        with open(pdf_path, "wb") as f:
            f.write(uploaded_file.getbuffer())

        # Extract tables from the PDF using tabula-py
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=False)

        if tables:
            # Convert the first table to DataFrame
            df = tables[0]

            # Save data to Excel
            excel_path = f'{save_directory}extracted_table.xlsx'
            df.to_excel(excel_path, index=False, sheet_name='Table')

            # Load the workbook and access the sheet
            wb = openpyxl.load_workbook(excel_path)
            ws = wb['Table']

            # Set the font and alignment for all cells to be the same
            for row in ws.iter_rows():
                for cell in row:
                    cell.font = Font(name='Calibri', size=11, bold=False)
                    cell.alignment = Alignment(horizontal='left', vertical='center')

            wb.save(excel_path)

            st.success("Table has been successfully converted to Excel.")
            st.download_button(label="Download Excel file", data=open(excel_path, "rb"), file_name="extracted_table.xlsx")

        else:
            # Convert PDF pages to images
            pages = convert_from_path(pdf_path, 300)
            for i, page in enumerate(pages):
                page.save(f'{save_directory}page_{i + 1}.jpg', 'JPEG')

            # Extract text from images
            extracted_texts = []
            for i in range(len(pages)):
                image_path = f'{save_directory}page_{i + 1}.jpg'
                text = pytesseract.image_to_string(Image.open(image_path))
                extracted_texts.append(text)

                # Detect images
                images = detect_images(image_path)
                numbered_images = [f'image_{image_counter + idx}' for idx, _ in enumerate(images)]
                for idx, image_name in enumerate(images):
                    if os.path.exists(image_name):
                        os.rename(image_name, f'{save_directory}{numbered_images[idx]}.jpg')
                image_data_list.append(numbered_images)
                image_counter += len(images)

            # Extract key-value pairs from text
            extracted_data_list = [extract_key_value_pairs(text) for text in extracted_texts]

            # Combine extracted data and images
            for i in range(len(extracted_data_list)):
                combined_data = extracted_data_list[i]
                combined_data['Images'] = ", ".join(image_data_list[i]) if image_data_list[i] else 'None'
                combined_data_list.append(combined_data)

            # Convert data to DataFrame
            df = pd.DataFrame(combined_data_list)

            # Save images as PDF
            all_image_paths = [item for sublist in image_data_list for item in sublist]
            pdf_files = save_images_as_pdf(all_image_paths)

            # Save data to Excel
            excel_path = f'{save_directory}extracted_customer_data.xlsx'
            with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                df.to_excel(writer, index=False, sheet_name='Data')

                # Load the workbook and access the sheet
                workbook = writer.book
                worksheet = workbook['Data']

                # Add hyperlinks to image PDFs in the "Images" column
                for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=worksheet.max_column, max_col=worksheet.max_column):
                    for cell in row:
                        if cell.value and cell.value != 'None':
                            image_links = cell.value.split(', ')
                            cell.value = ""
                            for img in image_links:
                                pdf_path = img.replace(".jpg", ".pdf")
                                cell.value += f'=HYPERLINK("{save_directory}{pdf_path}", "{pdf_path}")\\n'
                        cell.font = Font(color="0000FF", underline="single")

            workbook.save(excel_path)

            st.success("PDFs have been successfully converted to a single Excel file.")
            st.download_button(label="Download Excel file", data=open(excel_path, "rb").read(), file_name="extracted_customer_data.xlsx")
"""

with open("app.py", "w") as file:
    file.write(code)

# Run the Streamlit app with ngrok
from pyngrok import ngrok

# Set the authtoken
!ngrok authtoken 2k3M6V2JexDRZrKkzRRKSAi3GgF_7Y962v7jMydo5RBALvGSU

# Connect to ngrok
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py